# Classification Surrogate Tests

We are interested in testing whether or not a surrogate model can correctly identify unknown constraints based on categorical criteria with classification surrogates. Essentially, we want to account for scenarios where specialists can look at a set of experiments and label outcomes as 'acceptable', 'unacceptable', 'ideal', etc. 

This involves new models that produce `CategoricalOutput`'s rather than continuous outputs. Mathematically, if $g_{\theta}:\mathbb{R}^d\to[0,1]^c$ represents the function governed by learnable parameters $\theta$ which outputs a probability vector over $c$ potential classes (i.e. for input $x\in\mathbb{R}^d$, $g_{\theta}(x)^\top\mathbf{1}=1$ where $\mathbf{1}$ is the vector of all 1's) and we have acceptibility criteria for the corresponding classes given by $a\in\{0,1\}^c$, we can compute the scalar output $g_{\theta}(x)^\top a\in[0,1]$ which represents the expected value of acceptance as an objective value to be passed in as a constrained function.

In this script, we look at a modified and constrained version of the optimization problem associated with the [Levy function](https://www.sfu.ca/~ssurjano/levy.html), which has a global minima at $x^*=\mathbf{1}$. We classify constraints for three classes: 'acceptable', 'unacceptable', and 'ideal' based on how close we are to the optimal decision variable; obviously, this value is unknown in a real-world setting, but this serves as a reasonable example.

In [1]:
# Import packages
import bofire.strategies.api as strategies
from bofire.data_models.api import Domain, Outputs, Inputs
from bofire.data_models.features.api import ContinuousInput, ContinuousOutput, CategoricalOutput, CategoricalInput
from bofire.data_models.objectives.api import MinimizeObjective, MinimizeSigmoidObjective, ConstrainedCategoricalObjective
import numpy as np
import pandas as pd

c:\ProgramData\Anaconda3\envs\bofire-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Manual setup of the optimization domain

The following cells show how to manually setup the optimization problem in BoFire for didactic purposes.

In [2]:
# Write a function which scales the inputs according to the Levy function - i.e. computes $w_i$
def scale_inputs(x: pd.Series) -> pd.Series:
    return 1 + (x - 1) / 4

In [3]:
# Set-up the inputs and outputs, use categorical domain just as an example
input_features = Inputs(features=[ContinuousInput(key=f"x_{i}", bounds=(-2, 2)) for i in range(5)] + [CategoricalInput(key=f"x_5", categories=(0.0, 1.0))])

# here the minimize objective is used, if you want to maximize you have to use the maximize objective.
output_features = Outputs(features=[
        ContinuousOutput(key=f"f_{0}", objective=MinimizeObjective(w=1.)),
        CategoricalOutput(key=f"f_{1}", categories=("unacceptable", "acceptable", "ideal"), objective=ConstrainedCategoricalObjective(categories=("unacceptable", "acceptable", "ideal"), desirability=(False, True, True))), # This function will be associated with learning the categories
        ContinuousOutput(key=f"f_{2}", objective=MinimizeSigmoidObjective(w=1., tp=0.0, steepness=0.5)),
    ]
)

# Create domain
domain1 = Domain(inputs=input_features, outputs=output_features)

# Sample random points
sample_df = domain1.inputs.sample(50).astype(float) # Sample x's

# Write a function which outputs one continuous variable and another discrete based on some logic
sample_df["f_0"] = np.sin(np.pi * scale_inputs(sample_df["x_0"])) ** 2 + sum([(scale_inputs(sample_df[col]) - 1) ** 2 * (1 + 10 * np.sin(np.pi * scale_inputs(sample_df[col]) + 1) ** 2 if ind < len(sample_df.columns) else 1 + np.sin(2 * np.pi * scale_inputs(sample_df[col])) ** 2) for ind, col in enumerate(sample_df.columns)])
sample_df["f_1"] = "unacceptable"
sample_df.loc[sample_df[input_features.get_keys()].sum(1) >= 1.5, "f_1"] = "acceptable"
sample_df.loc[sample_df[input_features.get_keys()].sum(1) >= 3.5, "f_1"] = "ideal"
sample_df["f_2"] = sample_df["x_0"] + 1e-2 * np.random.uniform(size=(len(sample_df),))

sample_df.head(20)

,x_0,x_1,x_2,x_3,x_4,x_5,f_0,f_1,f_2
0,0.043387,0.471935,-1.903405,-1.505071,-1.631395,0.0,12.885008,unacceptable,0.049104
1,-0.516480,-0.519954,0.172009,-0.671419,0.576339,1.0,1.743262,unacceptable,-0.511997
2,1.420828,-0.997470,-0.869831,-0.603185,0.988815,0.0,2.214841,unacceptable,1.426101
3,-1.729386,-0.517152,0.792940,1.841196,-1.018597,1.0,6.757267,unacceptable,-1.723294
4,0.137756,-1.910416,0.266167,1.376514,1.853052,0.0,6.659112,acceptable,0.146894
5,0.615494,0.638574,-0.846471,0.486294,-1.844437,0.0,5.965467,unacceptable,0.623373
6,0.577691,-1.316748,-0.598852,1.499971,0.435526,1.0,2.791735,acceptable,0.586726
7,-1.382985,-1.629080,0.957606,1.263108,-1.248810,1.0,8.962357,unacceptable,-1.376337
8,-0.732977,-1.635727,-1.165820,-1.912441,1.095427,1.0,12.089185,unacceptable,-0.723748
9,-1.324801,-1.119786,1.021565,-0.304530,0.425360,1.0,4.633603,unacceptable,-1.322161


## Setup strategy and ask for candidates



In [4]:
from bofire.data_models.acquisition_functions.api import qEI
from bofire.data_models.strategies.api import SoboStrategy
from bofire.data_models.surrogates.api import BotorchSurrogates, MLPClassifierEnsemble, MixedSingleTaskGPSurrogate
from bofire.data_models.domain.api import Outputs

strategy_data = SoboStrategy(domain=domain1, 
                             acquisition_function=qEI(), 
                             surrogate_specs=BotorchSurrogates(surrogates=
                                    [
                                        MLPClassifierEnsemble(inputs=domain1.inputs, outputs=Outputs(features=[domain1.outputs.get_by_key("f_1")]), lr=0.1, n_epochs=100, hidden_layer_sizes=(20,10,)),
                                        MixedSingleTaskGPSurrogate(inputs=domain1.inputs, outputs=Outputs(features=[domain1.outputs.get_by_key("f_2")]))
                                    ]
                                )
                            )

strategy = strategies.map(strategy_data)

strategy.tell(sample_df)

In [5]:
candidates = strategy.ask(10)
candidates

c:\ProgramData\Anaconda3\envs\bofire-env\lib\site-packages\botorch\models\model.py:230: RuntimeWarning: Could not update `train_inputs` with transformed inputs since _MLPClassifierEnsemble does not have a `train_inputs` attribute. Make sure that the `input_transform` is applied to both the train inputs and test inputs.
  warnings.warn(
c:\ProgramData\Anaconda3\envs\bofire-env\lib\site-packages\botorch\models\model.py:230: RuntimeWarning: Could not update `train_inputs` with transformed inputs since _MLPClassifierEnsemble does not have a `train_inputs` attribute. Make sure that the `input_transform` is applied to both the train inputs and test inputs.
  warnings.warn(
c:\ProgramData\Anaconda3\envs\bofire-env\lib\site-packages\botorch\models\model.py:230: RuntimeWarning: Could not update `train_inputs` with transformed inputs since _MLPClassifierEnsemble does not have a `train_inputs` attribute. Make sure that the `input_transform` is applied to both the train inputs and test inputs.
  w

,x_0,x_1,x_2,x_3,x_4,x_5,f_0_pred,f_2_pred,f_1_pred,f_1_pred_unacceptable,f_1_pred_acceptable,f_1_pred_ideal,f_0_sd,f_2_sd,f_1_sd_unacceptable,f_1_sd_acceptable,f_1_sd_ideal,f_0_des,f_2_des,f_1_des
0,0.011502,-0.161920,1.972217,0.438555,0.457098,0.0,-0.508109,0.018091,ideal,0.005109,0.257733,0.737157,0.424743,0.002801,0.004573,0.346797,0.344908,0.508109,0.497739,0.994891
1,0.165405,-0.124200,-0.283804,0.356975,0.652289,1.0,-0.549702,0.171525,acceptable,0.001980,0.996553,0.001467,0.365422,0.002720,0.001716,0.003879,0.002428,0.549702,0.478572,0.998020
2,-0.008095,-0.188512,1.662439,0.346688,0.430895,1.0,-0.371163,-0.001649,acceptable,0.000494,0.578223,0.421283,0.418334,0.002777,0.000678,0.484275,0.484395,0.371163,0.500206,0.999506
3,-0.034115,2.000000,-0.095091,-0.378793,2.000000,0.0,0.887337,-0.029129,ideal,0.001376,0.136712,0.861912,0.899251,0.002871,0.003050,0.303232,0.302495,-0.887337,0.503641,0.998624
4,-0.011600,0.117177,1.615437,0.201891,0.744478,0.0,-0.337936,-0.005347,ideal,0.007744,0.236033,0.756223,0.426527,0.002773,0.007903,0.379799,0.377517,0.337936,0.500668,0.992256
5,-0.073124,-0.243746,2.000000,0.402120,0.352889,1.0,-0.395214,-0.066696,acceptable,0.000355,0.507754,0.491891,0.408950,0.002803,0.000558,0.472051,0.472281,0.395214,0.508336,0.999645
6,0.096887,0.753092,-0.296234,-0.268697,1.757006,1.0,0.256462,0.102279,acceptable,0.000452,0.614570,0.384978,0.509006,0.002777,0.000830,0.464693,0.464979,-0.256462,0.487218,0.999548
7,0.173888,0.237557,-0.316069,-0.011089,0.894507,1.0,-0.376064,0.179808,acceptable,0.001598,0.996005,0.002396,0.405710,0.002727,0.001953,0.005998,0.004370,0.376064,0.477539,0.998402
8,-0.249681,-0.060291,2.000000,0.359377,0.670042,0.0,-0.281907,-0.243816,ideal,0.005539,0.221772,0.772690,0.454217,0.002804,0.005596,0.310703,0.308060,0.281907,0.530439,0.994461
9,0.126138,-0.276652,-0.165111,0.671939,0.442778,1.0,-0.436071,0.132389,acceptable,0.002137,0.996403,0.001460,0.346124,0.002722,0.001572,0.003672,0.002423,0.436071,0.483457,0.997863


## See performance of the classifier

In [6]:
print(f"We defined 'unacceptable' as values in (-infinity, 1.5), 'acceptable' as values in [1.5, 3.5), and 'ideal' as values in [3.5, infinity)\n")
pd.concat((candidates[[feat.key for feat in input_features]].astype(float).sum(1), candidates["f_1_pred"]), axis=1)

We defined 'unacceptable' as values in (-infinity, 1.5), 'acceptable' as values in [1.5, 3.5), and 'ideal' as values in [3.5, infinity)



,0,f_1_pred
0,2.717452,ideal
1,1.766665,acceptable
2,3.243415,acceptable
3,3.492002,ideal
4,2.667383,ideal
5,3.438138,acceptable
6,3.042054,acceptable
7,1.978794,acceptable
8,2.719447,ideal
9,1.799092,acceptable
